In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
data = pd.read_csv('./vk_data.tsv', sep='\t')

In [3]:
data.head(3)

,text,date,likes,comments,reposts
0,"В Паблике, в тестовом режиме, запущено некое н...",2017-03-29 11:44,223,2,191
1,#Ищутебя_поиск Админ опубликуй пожалуйста Ищу ...,2017-04-22 12:34,1,0,0
2,"#Ищутебя_парни Ищу девушку. О себе: 20 лет, ро...",2017-04-22 12:33,0,0,0


In [11]:
from nltk import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import RussianStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [12]:
def tokenize(text):
    rus_stemmer = RussianStemmer()
    text = text.lower()
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(text)
    tokens = ' '.join(tokens)
    return rus_stemmer.stem(tokens)

In [13]:
tokenized_texts = data['text'].apply(tokenize)

In [40]:
count_vectorizer = CountVectorizer(min_df=5)
bag_of_words = count_vectorizer.fit_transform(tokenized_texts)

In [27]:
t = ['это тест', 'и еще один']
count_vectorizer.fit_transform(t).toarray()

array([[0, 0, 1, 1],
       [1, 1, 0, 0]], dtype=int64)

In [41]:
bag_of_words.shape

(42887, 16701)

In [46]:
train_ind = int(data.shape[0] * 0.8)

In [52]:
lr = LinearRegression(n_jobs=-1)

In [53]:
lr.fit(bag_of_words[:train_ind], data['likes'][:train_ind])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=-1, normalize=False)

In [50]:
lr.score(bag_of_words[:train_ind], data['likes'][:train_ind])

0.81870651908177883

In [54]:
lr.score(bag_of_words[train_ind:], data['likes'][train_ind:])

-10.093546101778317

In [56]:
from sklearn.model_selection import cross_val_score

In [57]:
lr_cv = cross_val_score(lr, bag_of_words[:train_ind], data['likes'][:train_ind],
                n_jobs=-1, cv=5).mean()

In [61]:
np.sqrt(np.sum((data['likes'][train_ind:] -
                np.mean(data['likes'][train_ind:])) ** 2 /
               len(data['likes'][train_ind:])))

25.032884657163496

In [62]:
from sklearn.linear_model import ElasticNet

In [63]:
X_train, y_train = bag_of_words[:train_ind], data['likes'][:train_ind]
X_test, y_test = bag_of_words[train_ind:], data['likes'][train_ind:]

In [65]:
enet = ElasticNet(random_state=42, )
enet.fit(X_train, y_train)

ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=42, selection='cyclic', tol=0.0001, warm_start=False)

In [66]:
enet.score(X_train, y_train)

0.25816144013201103

In [67]:
enet.score(X_test, y_test)

-0.81543395156641041

In [69]:
from sklearn.metrics import mean_squared_error

In [70]:
mean_squared_error(y_test, enet.predict(X_test))

1137.6331790954923

In [71]:
mean_squared_error(y_train, enet.predict(X_train))

3116.6371961132527

In [72]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
enet = ElasticNet(normalize=True, )
param_dist = {"alpha": [0.0, 0.5, 1.0],
              "l1_ratio": [0.0, 0.25, 0.5, 0.75, 1.0],
              "tol": [0.00001, 0.001, 0.1, 1]}

model = RandomizedSearchCV(enet, param_distributions=param_dist,
                          n_iter=30, verbose=1, random_state=42)

model.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


D:\soft\dev\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:238: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  estimator.fit(X_train, y_train, **fit_params)
